# Inferring
In this lesson, you will infer sentiment and topics from product reviews and news articles.

## Setup

In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [3]:
lamp_review = """
Precisava de uma luminária legal para o meu quarto, e essa tinha \
armazenamento adicional e um preço não muito alto. \
Entendi rápido. O barbante da nossa lâmpada quebrou durante o \
trânsito e a empresa enviou alegremente um novo. \
Veio dentro de alguns dias também. Foi fácil colocar \
junto. Eu tinha uma peça faltando, então entrei em contato com o \
suporte e eles rapidamente me deram a peça que faltava! \
A Lumina me parece ser uma grande empresa que se importa \
sobre seus clientes e produtos!!
"""

## Sentiment (positive/negative)

In [4]:
prompt = f"""
Qual é o sentimento da seguinte análise do produto,
que é delimitado com backticks triplos?

Texto da revisão: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

O sentimento da análise é positivo e satisfeito com o produto e o atendimento da empresa.


In [5]:
prompt = f"""
Qual é o sentimento da seguinte análise do produto,
que é delimitado com backticks triplos?

Dê sua resposta como uma única palavra, seja "positivo" \
ou "negativo".

Texto da revisão:'''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

Positivo.


## Identify types of emotions

In [6]:
prompt = f"""
Identifique uma lista de emoções que o escritor do \
seguinte revisão está expressando. Não inclua mais de \
cinco itens na lista. Formate sua resposta como uma lista de \
palavras minúsculas separadas por vírgulas.

Texto da revisão: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

satisfeito, feliz, grato, impressionado, confiante


In [7]:
prompt = f"""
O autor da resenha a seguir está expressando raiva?\
A revisão é delimitada por acentos graves triplos. \
Dê sua resposta como sim ou não.

Texto da revisão:'''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

Não.


## Extract product and company name from customer reviews

In [8]:
prompt = f"""
Identifique os seguintes itens do texto de revisão:
- Item comprado pelo revisor
- Empresa que fez o item

A revisão é delimitada por acentos graves triplos. \
Formate sua resposta como um objeto JSON com \
"Item" e "Marca" como chaves.
Se a informação não estiver presente, use "desconhecido" \
como o valor.
Faça sua resposta o mais curta possível.
  
Texto da revisão: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
  "Item": "luminária",
  "Marca": "Lumina"
}


## Doing multiple tasks at once

In [9]:
prompt = f"""
Identifique os seguintes itens do texto de revisão:
- Sentimento (positivo ou negativo)
- O revisor está expressando raiva? (verdadeiro ou falso)
- Item comprado pelo revisor
- Empresa que fez o item

A revisão é delimitada por acentos graves triplos. \
Formate sua resposta como um objeto JSON com \
"Sentiment", "Anger", "Item" e "Brand" como as chaves.
Se a informação não estiver presente, use "desconhecido" \
como o valor.
Faça sua resposta o mais curta possível.
Formate o valor Anger como um booleano.

Texto da revisão:'''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
  "Sentiment": "positivo",
  "Anger": false,
  "Item": "luminária com armazenamento adicional",
  "Brand": "Lumina"
}


## Inferring topics

In [10]:
story = """
Em uma pesquisa recente realizada pelo governo,
funcionários do setor público foram solicitados a avaliar seu nível
de satisfação com o departamento em que trabalham.
Os resultados revelaram que a NASA foi o mais popular
departamento com um índice de satisfação de 95%.

Um funcionário da NASA, John Smith, comentou sobre as descobertas,
afirmando: "Não estou surpreso que a NASA saiu por cima.
É um ótimo lugar para trabalhar com pessoas incríveis e
oportunidades incríveis. tenho orgulho de fazer parte
uma organização tão inovadora."

Os resultados também foram bem recebidos pela equipe de gerenciamento da NASA,
com o diretor Tom Johnson afirmando: "Estamos entusiasmados em
ouvir que nossos funcionários estão satisfeitos com seu trabalho na NASA.
Temos uma equipe talentosa e dedicada que trabalha incansavelmente
para alcançar nossos objetivos, e é fantástico ver que seus
o trabalho árduo está valendo a pena."

A pesquisa também revelou que o
Administração da Previdência Social teve a menor satisfação
classificação, com apenas 45% dos funcionários indicando que eram
satisfeitos com seu trabalho. O governo se comprometeu a
abordar as preocupações levantadas pelos funcionários na pesquisa e
trabalhar para melhorar a satisfação no trabalho em todos os departamentos.
"""

## Infer 5 topics

In [11]:
prompt = f"""
Determine cinco tópicos que estão sendo discutidos no \
seguinte texto, que é delimitado por crases triplos.

Faça com que cada item tenha uma ou duas palavras.

Formate sua resposta como uma lista de itens separados por vírgulas.

Exemplo de texto: '''{story}'''
"""
response = get_completion(prompt)
print(response)

1. Pesquisa de satisfação no setor público
2. Departamentos com maior e menor índice de satisfação
3. Comentários de funcionários da NASA sobre a pesquisa
4. Reação da equipe de gerenciamento da NASA aos resultados
5. Compromisso do governo em melhorar a satisfação no trabalho em todos os departamentos.


In [12]:
response.split(sep=',')

['1. Pesquisa de satisfação no setor público\n2. Departamentos com maior e menor índice de satisfação\n3. Comentários de funcionários da NASA sobre a pesquisa\n4. Reação da equipe de gerenciamento da NASA aos resultados\n5. Compromisso do governo em melhorar a satisfação no trabalho em todos os departamentos.']

In [13]:
topic_list = [
    "nasa", "local government", "engineering", 
    "employee satisfaction", "federal government"
]

## Make a news alert for certain topics

In [14]:
prompt = f"""
Determine se cada item na seguinte lista de \
tópicos é um tópico no texto abaixo, que
é delimitado com acentos graves triplos.

Dê sua resposta como uma lista com 0 ou 1 para cada tópico.\

Lista de tópicos: {", ".join(topic_list)}

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

nasa: 1
local government: 0
engineering: 0
employee satisfaction: 1
federal government: 1


In [15]:
topic_dict = {i.split(': ')[0]: int(i.split(': ')[1]) for i in response.split(sep='\n')}
if topic_dict['nasa'] == 1:
    print("ALERT: New NASA story!")

ALERT: New NASA story!


## Try experimenting on your own!